In [28]:
# Importar biblioteca pandas para leitura da base
import pandas as pd
import numpy as np

# Ler base de dados e atribuir interrogação para os valores nulos
df = pd.read_csv('adult.data', na_values=' ?')

# Mostra os 5 primeiros dados da base
df.head()
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
ocupation         1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
greater              0
dtype: int64

In [29]:
df = df.dropna()

# Mostrando a forma (Linhas x Colunas) após remover as linhas vazias
print(df.shape)

(30162, 15)


preparacao

In [30]:
df['sex'] = df.sex.map({" Male": 1, " Female": 0})

In [31]:
df['workclass_private'] = ''
df['workclass_others'] = ''

In [32]:
df['workclass_private'] = df['workclass'] == ' Private'
df['workclass_others'] = df['workclass'] != ' Private'

In [33]:
df['greater'] = df.greater.map({' <=50K': 0, ' >50K': 1})

In [39]:
from sklearn.preprocessing import StandardScaler
usedColumns = [
  # 'age',
  'workclass_private',
  'workclass_others',
  'fnlwgt','education',
  'education-num',
  'marital-status',
  'ocupation','relationship','race','sex','hours-per-week', 'greater']
dfDesired = df[usedColumns]
df_c = pd.get_dummies(dfDesired, columns=['education','marital-status','ocupation', 'relationship', 'race'])
df_c.head()
# scaler = StandardScaler()
# scaler.fit(df_c)
# df_c = scaler.transform(df_c)

,workclass_private,workclass_others,fnlwgt,education-num,sex,hours-per-week,greater,education_ 10th,education_ 11th,education_ 12th,...,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White
0,False,True,77516,13,1,40,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,False,True,83311,13,1,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,True,False,215646,9,1,40,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,True,False,234721,7,1,40,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,True,False,338409,13,0,40,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [40]:
from sklearn.utils import shuffle
train_percentage = int(len(df)*70/100)
print(train_percentage)

df_shuffled = shuffle(df_c, random_state=42)

21113


holdout

In [41]:
y = df_shuffled['greater']
X_train, X_test = df_shuffled[:train_percentage], df_shuffled[train_percentage:]
y_train, y_test = y[:train_percentage], y[train_percentage:]


KNN

In [51]:
from sklearn.neighbors import KNeighborsClassifier
 
# Aplicação do modelo KNN e treinamento do mesmo
modelo = KNeighborsClassifier(n_neighbors = 13)
modelo.fit(X_train, y_train)
 
# Aplicação da predição no modelo
y_pred = modelo.predict(X_test)


In [52]:
from sklearn.metrics import accuracy_score
 
# Demonstração da acurácia encontrada pelo modelo
accuracy_score(y_pred, y_test)


0.7406343242347221

In [53]:
from sklearn.metrics import classification_report
 
# Exibindo relatório da precisão de acertos dos valores menores e maiores que 50k
print(classification_report(y_pred, y_test))


              precision    recall  f1-score   support

           0       0.97      0.76      0.85      8697
           1       0.06      0.38      0.10       352

    accuracy                           0.74      9049
   macro avg       0.51      0.57      0.48      9049
weighted avg       0.93      0.74      0.82      9049



In [54]:
from sklearn.model_selection import GridSearchCV
 
# Definir o intervalo de parametros para teste
params = {'n_neighbors':[1,5,9,13,101,207]}
clf = GridSearchCV(modelo, params)
clf.fit(X_train, y_train)


GridSearchCV(estimator=KNeighborsClassifier(n_neighbors=13),
             param_grid={'n_neighbors': [1, 5, 9, 13, 101, 207]})

In [55]:
display(pd.DataFrame(clf.cv_results_))


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008896,0.001761,0.876707,0.014152,1,{'n_neighbors': 1},0.663036,0.673455,0.659484,0.654429,0.678115,0.665704,0.008798,6
1,0.007911,0.000387,1.200530,0.040006,5,{'n_neighbors': 5},0.709922,0.711579,0.704002,0.712932,0.709379,0.709563,0.003050,5
2,0.008337,0.000262,1.213879,0.022907,9,{'n_neighbors': 9},0.727445,0.727682,0.730523,0.733065,0.729986,0.729740,0.002061,4
3,0.008089,0.000372,1.209944,0.029767,13,{'n_neighbors': 13},0.737390,0.739285,0.740469,0.740407,0.739697,0.739450,0.001121,3
4,0.008340,0.000424,1.234984,0.034818,101,{'n_neighbors': 101},0.751598,0.751598,0.751598,0.751776,0.751776,0.751670,0.000087,1
5,0.008303,0.000561,1.279258,0.026282,207,{'n_neighbors': 207},0.751598,0.751598,0.751598,0.751776,0.751776,0.751670,0.000087,1


In [56]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
 
# Atribuição de valor de X e y
X = df_c
y = df["greater"].values
 
# Insere as informação desejadas para a realização da Validação Cruzada, no caso, com 15 divisões
kf = KFold(n_splits = 15, random_state = 42, shuffle = True)
 
# Inicializa vetor de scores, responsável por armazenar dados de cada iteração da Validação Cruzada
scores = []
 
# Percorre cada iteração da Validação Cruzada
for train_index, test_index in kf.split(X):
    
    # Define os valores de treino e teste, baseado com a iteração da Validação Cruzada
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Aplicação do modelo KNN e treinamento do mesmo
    modelo = KNeighborsClassifier(n_neighbors = 13)
    modelo.fit(X_train, y_train)
    # Aplicação da predição no modelo
    y_pred = modelo.predict(X_test)
    # Armazena os valores de cada iteração da Validação Cruzada
    score = modelo.score(X_test, y_test)
    scores.append(score)
 
# Transforma em array os valores dos scores de forma padronizada
scores = np.array(scores)
 
# Printa a média e o desvio da padrão dos modelos encontrados, além de printar os scores encontrados


KeyError: "None of [Int64Index([    0,     1,     2,     3,     4,     5,     7,     8,     9,\n               10,\n            ...\n            30152, 30153, 30154, 30155, 30156, 30157, 30158, 30159, 30160,\n            30161],\n           dtype='int64', length=28151)] are in the [columns]"